In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams
import os
import re
%matplotlib inline

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import jellyfish

# import torch
# from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('stsb-roberta-large')

# Imported the inaugural corpus from nltk.corpus
import nltk
from nltk.corpus import inaugural
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

import gensim
from gensim.utils import simple_preprocess
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import num2words
from num2words import num2words
import time
# from time import time

pd.options.display.max_colwidth = 10000

In [2]:
masterkey_pdtdesc = pd.read_csv('~/Downloads/mapped_unique_masterkey_product_desc.csv', engine='python')

In [3]:
masterkey_pdtdesc.sort_values(by = 'PRODUCT_DESCRIPTION', inplace = True)

In [4]:
invoice_lessthan10_UPC_desc = pd.read_csv('~/Downloads/lessthan10_UPC_desc.csv', engine='python')

In [5]:
invoice_lessthan10_UPC_desc.sort_values(by = 'INVOICE_PACKAGE_DESCRIPTION', inplace = True)

In [6]:
subset_invoice = invoice_lessthan10_UPC_desc[:100000]

In [7]:
subset_master = masterkey_pdtdesc

In [8]:
subset_master["joined"] = subset_master['PRODUCT_MASTER_CATALOG_KEY'].map(str) + " |" + subset_master['PRODUCT_DESCRIPTION'] 

In [9]:
# def partial_ratio(tup):
#     return pd.Series([fuzz.partial_ratio(*tup)] ,['partial'])

# def token_set_ratio(tup):
#     return pd.Series([fuzz.token_set_ratio(*tup)],['set_ratio'])

In [10]:
start = time.time()

In [11]:
# def logic(invoice_desc,ct,n):
    
    

#     l =[]
#     l.append(invoice_desc)

# #Multiindex and reindexing
#     compare = pd.MultiIndex.from_product([l, subset_master['PRODUCT_DESCRIPTION']]).to_series()
#     df_compare = compare.reset_index()    
# #Renaming the columns 
   
#     df_compare.rename(columns={ df_compare.columns[0]: "INVOICE_PACKAGE_DESCRIPTION" }, inplace = True)
#     df_compare.rename(columns={ df_compare.columns[2]: "concat" }, inplace = True)
#     df_compare.dropna(axis='rows',inplace = True)
# #Call the function for calculating ratios

#     df_compare["partial"] = df_compare["concat"].apply(partial_ratio)
#     df_compare["token_set_ratio"] = df_compare["concat"].apply(token_set_ratio)
    
#     return df_compare.sort_values(by =['partial','token_set_ratio'],ascending=False).head(10)
    

    
    
    
    

In [12]:
def partial_ratio2(tup):
    l=[]
    l.append(tup)
  
    sep = ''
    out = sep.join(tup[1])
    n = out.split('|')
    
    return pd.Series([fuzz.partial_ratio(tup[0],n[1].strip())] ,['partial'])

def token_set_ratio2(tup):
    l=[]
    l.append(tup)
    sep = ''
    out = sep.join(tup[1])
    n = out.split('|')
  
    return pd.Series([fuzz.token_set_ratio(tup[0],n[1].strip())],['set_ratio'])

In [13]:
def logic2(invoice_desc,ct,n):
    
    

    l =[]
    l.append(invoice_desc)

#Multiindex and reindexing
    compare = pd.MultiIndex.from_product([l, subset_master['joined']]).to_series()
    df_compare = compare.reset_index()    
#Renaming the columns 
   
    df_compare.rename(columns={ df_compare.columns[0]: "INVOICE_PACKAGE_DESCRIPTION" }, inplace = True)
    df_compare.rename(columns={ df_compare.columns[2]: "concat" }, inplace = True)
    df_compare.dropna(axis='rows',inplace = True)
#Call the function for calculating ratios

    df_compare["partial"] = df_compare["concat"].apply(partial_ratio2)
    df_compare["token_set_ratio"] = df_compare["concat"].apply(token_set_ratio2)
    
    return df_compare.sort_values(by =['partial','token_set_ratio'],ascending=False).head(10)
    

    
    
    
    

In [ ]:

n = 0
ct =0
for i in subset_invoice.index:
    ct += 1

    x = subset_invoice.loc[i, "INVOICE_PACKAGE_DESCRIPTION"]
    
    res = logic2(x,ct,n)

    export_file = res[['INVOICE_PACKAGE_DESCRIPTION','joined','partial','token_set_ratio']]
    file1 = open("unmatched_fuzzymatch_top10_ratios{}.csv".format(n), "a") 
    export_file.to_csv(file1, header=False)
    file1.close()
    if ct == 200:
        print(ct)
    if ct == 10000:
        print("--->",n)
        n += 1
        del res
        ct = 0

200


In [ ]:
end = time.time()
print("time taken: " ,(end-start))

In [ ]:
ss1hj

In [ ]:
# df_compare.rename(columns={ df_compare.columns[2]: "concat" }, inplace = True)
#     df_compare.dropna(axis='rows',inplace = True)
# #calling partial model for score     
#     df_compare['partial'], df_compare['set_ratio'] = df_compare['concat'].apply(partial)
#     df_compare.rename(columns={ df_compare.columns[0]: "INVOICE_PACKAGE_DESCRIPTION" }, inplace = True)
#     res = df_compare.sort_values('partial',ascending=False).head(10)
    
#     if ct == 2:
#         print("--->",n)
#         n += 1
#         export_file = res[['INVOICE_PACKAGE_DESCRIPTION','PRODUCT_DESCRIPTION','partial','set_ratio']]
#         file1 = open("N_unmatched_fuzzymatch_top10_ratios{}.csv".format(n), "a") 
#         export_file.to_csv(file1, header=False)
#         file1.close()
#         del df_compare
#         del res
#         ct = 0